In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.optimizers import Adam
import numpy as np
from glob import glob

2024-03-04 14:04:21.808184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 14:04:21.808295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 14:04:21.939062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [4]:
img_size = [224, 224]

In [5]:
train_path = '/kaggle/input/rice-dataset-updated/Rice Dataset/Train'
test_path = '/kaggle/input/rice-dataset-updated/Rice Dataset/Test'

In [6]:
efficientnet = EfficientNetB7(input_shape=img_size + [3], weights='imagenet', include_top=False)

258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [7]:
for layer in efficientnet.layers[:-20]:
    layer.trainable = False
for layer in efficientnet.layers[-20:]:
    layer.trainable = True

In [8]:
folders = glob('/kaggle/input/rice-dataset-updated/Rice Dataset/Train/*')

In [9]:
x = Flatten()(efficientnet.output)
x = Dense(1024, activation='relu')(x)  # Additional dense layer
prediction = Dense(len(folders), activation='softmax')(x)

In [10]:
model = Model(inputs=efficientnet.input, outputs=prediction)

In [11]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │      1,728 │ stem_conv_pad[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      2,048 │ block1a_se_excit

 Total params: 192,554,396 (734.54 MB)

 Trainable params: 136,299,429 (519.94 MB)

 Non-trainable params: 56,254,967 (214.60 MB)

In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [14]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size[0], img_size[1]),
    batch_size=32,
    class_mode='categorical'
)

Found 10000 images belonging to 5 classes.


In [16]:
test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size[0], img_size[1]),
    batch_size=32,
    class_mode='categorical'
)

Found 2500 images belonging to 5 classes.


In [17]:
model.fit(
    training_set,
    validation_data=test_set,
    epochs=20,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1709561602.659465     231 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1709561602.846214     231 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 866ms/step - accuracy: 0.6502 - loss: 2.9216

W0000 00:00:1709561891.465490     232 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 485s 1s/step - accuracy: 0.6506 - loss: 2.9158 - val_accuracy: 0.9036 - val_loss: 0.2470
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20


W0000 00:00:1709561922.370407     231 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


313/313 ━━━━━━━━━━━━━━━━━━━━ 137s 429ms/step - accuracy: 0.8472 - loss: 0.3995 - val_accuracy: 0.9304 - val_loss: 0.2063
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 118us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 137s 427ms/step - accuracy: 0.8748 - loss: 0.3421 - val_accuracy: 0.9192 - val_loss: 0.2069
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 136us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 138s 431ms/step - accuracy: 0.8889 - loss: 0.2954 - val_accuracy: 0.9248 - val_loss: 0.1940
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 128us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 146s 457ms/step - accuracy: 0.9006 - loss: 0.2653 - val_accuracy: 0.9512 - val_loss: 0.1454
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [18]:
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print('\nTest accuracy:', test_acc)

79/79 - 13s - 161ms/step - accuracy: 0.9488 - loss: 0.1351

Test accuracy: 0.9488000273704529


In [19]:
model.save('rice_model_efficientnetb7.h5')

In [20]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/rice_model_efficientnetb7.h5')

In [21]:
import pickle
class_indices = training_set.class_indices

with open('class_indices.pkl', 'wb') as file:
    pickle.dump(class_indices, file)

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import pickle

model_path = '/kaggle/working/rice_model_efficientnetb7.h5'
model = load_model(model_path)

with open('/kaggle/working/class_indices.pkl', 'rb') as f:
    class_indices = pickle.load(f)
class_names = {v: k for k, v in class_indices.items()}

def prepare_image(file_path, img_size):
    img = image.load_img(file_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

img_path = '/kaggle/input/222222222/2.jpg'

prepared_img = prepare_image(img_path, img_size=[224, 224])

prediction = model.predict(prepared_img)
predicted_class_index = np.argmax(prediction, axis=1)
predicted_class_name = class_names[predicted_class_index[0]]

print(f"Predicted class name: {predicted_class_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
Predicted class name: Basmati
